<a href="https://www.kaggle.com/code/johnmwadime/chatbot?scriptVersionId=97905879" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

#### <p>We will be installing python libraries; 
<ul><li>nltk</li> <li>NumPy</li><li>gTTs (google text-to-speech)</li><li>mpg123 and portaudio, for accessing the microphone from the system</li><li>SpeechRecognition</li><li>scikit-learn</li></ul></p>

In [ ]:
 #!pip install --upgrade pip
#!pip install --upgrade setuptools
#pip install SpeechRecognition numpy gTTs sklearn
#!pip install pyAudioAnalysis
#!pip install PyAudio
#!pip install portaudio
#pip install mpg123

#### <p>Importing neccessary libraries and modules</p> 

In [ ]:
#importing necessary libraries
import io
import random
import string
import warnings
import mpg123
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import warnings
from gtts import gTTS #text to speech conversion
import os 
warnings.filterwarnings('ignore')
import speech_recognition as sr 
import nltk
from nltk.stem import WordNetLemmatizer
#for downloading package files can be commented after First run
#nltk.download('popular', quiet=True)
#nltk.download('nps_chat',quiet=True)
#nltk.download('punkt') 
#nltk.download('wordnet')
#intro_join ="../input/intro-join/intro_join.txt"

#### <p>We will write a function to classify user input, which uses nps_chat corpora and naive Bayes classifier to categorize the input type by classifying them into listed categories.Where else Bot will answer only to Question type classes.</p>

In [ ]:
posts = nltk.corpus.nps_chat.xml_posts()[:10000]
# To Recognise input type as QUES. 
def dialogue_act_features(post):
    features = {}
    for word in nltk.word_tokenize(post):
        features['contains({})'.format(word.lower())] = True
    return features
featuresets = [(dialogue_act_features(post.text), post.get('class')) for post in posts]
size = int(len(featuresets) * 0.1)
train_set, test_set = featuresets[size:], featuresets[:size]
classifier = nltk.NaiveBayesClassifier.train(train_set)

#### <p>To make our chatbot more engaging and interactive, we will create a greeting function.</p>

In [ ]:
# Keyword Matching
GREETING_INPUTS = ("hello","heey", "hi", "greetings", "sup", "what's up","hey",)
GREETING_RESPONSES = ["hi", "hey", "*nods*", "hi there", "hello", "I am glad! You are talking to me"]
def greeting(sentence):
    """If user's input is a greeting, return a greeting response"""
    for word in sentence.split():
        if word.lower() in GREETING_INPUTS:
            return random.choice(GREETING_RESPONSES)

#### <p>We will call Corpus of our chatbot and perform some NLP pre-processing steps on it, i.e., Sentence and Word Tokenization, Lemmatization, Normalisation. We are calling our Corpus as intro_join. </p>

In [ ]:
#Reading in the input_corpus
with open('../input/intro-join/intro_join.txt','r', encoding='utf8', errors ='ignore') as fin:
    raw = fin.read().lower()
#TOkenisation
sent_tokens = nltk.sent_tokenize(raw)# converts to list of sentences 
word_tokens = nltk.word_tokenize(raw)# converts to list of words
# Preprocessing
lemmer = WordNetLemmatizer()
def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

#### <p>To make it look appealing, we will add the color pallet, which is a list of functions using the format command, for adding color to std terminal output.</p>

In [ ]:
#colour palet
def prRed(skk): print("\033[91m {}\033[00m" .format(skk)) 
def prGreen(skk): print("\033[92m {}\033[00m" .format(skk)) 
def prYellow(skk): print("\033[93m {}\033[00m" .format(skk)) 
def prLightPurple(skk): print("\033[94m {}\033[00m" .format(skk)) 
def prPurple(skk): print("\033[95m {}\033[00m" .format(skk)) 
def prCyan(skk): print("\033[96m {}\033[00m" .format(skk)) 
def prLightGray(skk): print("\033[97m {}\033[00m" .format(skk)) 
def prBlack(skk): print("\033[98m {}\033[00m" .format(skk))

#### <p>Now we will create a function for processing the user response converting it into a vectorized form and get the best result from Corpus via computing TF-IDF cosine similarity between the question and answer.</p>

In [ ]:
# Generating response and processing 
def response(user_response):
    robo_response=''
    sent_tokens.append(user_response)
    TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english')
    tfidf = TfidfVec.fit_transform(sent_tokens)
    vals = cosine_similarity(tfidf[-1], tfidf)
    idx=vals.argsort()[0][-2]
    flat = vals.flatten()
    flat.sort()
    req_tfidf = flat[-2]
    if(req_tfidf==0):
        robo_response=robo_response+"I am sorry! I don't understand you"
        return robo_response
    else:
        robo_response = robo_response+sent_tokens[idx]
        return robo_response

#### <p>Now we are almost ready to print the First message from Bot 😃</p>

In [ ]:
#Recording voice input using microphone 
file = "./file.mp3"
flag=True
fst="My name is John. I will answer your inquiries about Python. Welcome! "
tts = gTTS(fst, lang= language, slow= False)
tts.save(file)
os.system(file )
r = sr.Recognizer()
prYellow(fst)

#### <p>It is using speech recognition for registering user input using the microphone. Converting it into text form, Searching for its answers from the processed Corpus, and returning the output using text-to-speech. It will continue taking user input and answering until the user says Bye/Goodbye.</p>

In [ ]:

while(flag==True):
    with sr.Microphone() as source:
        audio= r.listen(source)
    try:
        user_response = format(r.recognize(audio))
        print("\033[91m {}\033[00m" .format("YOU SAID : "+user_response))
    except sr.UnknownValueError:
        prYellow("Oops! Didn't catch that")
        pass
    
    #user_response = input()
    #user_response=user_response.lower()
    clas=classifier.classify(dialogue_act_features(user_response))
    if(clas!='Bye'):
        if(clas=='Emotion'):
            flag=False
            prYellow("John: You are welcome..")
        else:
            if(greeting(user_response)!=None):
                print("\033[93m {}\033[00m" .format("John: "+greeting(user_response)))
            else:
                print("\033[93m {}\033[00m" .format("John: ",end=""))
                res=(response(user_response))
                prYellow(res)
                sent_tokens.remove(user_response)
                tts = gTTS(res, lang = language,slow = False)
                tts.save(file)
                os.system(file)
    else:
        flag=False

### YET to explain every part of my code. Meanwhile, i need someone to brainstorm with me on how to install portaudio and pyaudio  for my notebook